#Installs

In [2]:
# Install required libraries
!apt-get update
!apt-get install -y bedtools  # Install BEDTools
!pip install matplotlib_venn
!pip install pybedtools
!pip install upsetplot

import pandas as pd
import numpy as np
from itertools import product
from upsetplot import UpSet, from_indicators
from google.colab import files
from pybedtools import BedTool
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from matplotlib_venn import venn3
from google.colab import drive

# Mount Google Drive (if needed for saving outputs)
#drive.mount('/content/drive')

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,104 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,103 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,758 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,266 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-update

#2-way

In [ ]:
#2way

# Upload BED files for the two transcription factors (use the raw peaks.bed files, not anno_peaks or anno_genes)
# IMPORTANT: Use your "Condition1_TF-A_peaks.bed" and "Condition1_TF-B_peaks.bed" files
uploaded = files.upload()
bed_files = list(uploaded.keys())  # Order should be: TF-A peaks first, then TF-B peaks --------------------------------------------------------------------

# Load BED files as BedTool objects and sort them
# (Sorting is a good idea to ensure consistent output from Bedtools.)
a_bed = BedTool(bed_files[0]).sort()
b_bed = BedTool(bed_files[1]).sort()

# Identify shared and unique peaks (with at least 1 bp overlap)
# "u=True" returns peaks in A that have any overlap in B (i.e. shared)
# "v=True" returns peaks in A that do NOT overlap with B (i.e. unique)
shared_bed = a_bed.intersect(b_bed, u=True)
unique_a_bed = a_bed.intersect(b_bed, v=True)
unique_b_bed = b_bed.intersect(a_bed, v=True)

# Count peaks in each category
unique_a = unique_a_bed.count()
unique_b = unique_b_bed.count()
shared_peaks = shared_bed.count()

total_a = unique_a + shared_peaks
total_b = unique_b + shared_peaks

# Print counts for validation
print(f"Unique TF-A peaks: {unique_a}")
print(f"Unique TF-B peaks: {unique_b}")
print(f"Shared peaks: {shared_peaks}")
print(f"Total TF-A peaks: {total_a}")
print(f"Total TF-B peaks: {total_b}")

# Calculate percentages for additional info in the plot (optional)
perc_a_unique = (unique_a / total_a) * 100  # TF-A unique percentage
perc_a_shared = (shared_peaks / total_a) * 100  # TF-A shared percentage
perc_b_unique = (unique_b / total_b) * 100  # TF-B unique percentage
perc_b_shared = (shared_peaks / total_b) * 100  # TF-B shared percentage

# Plot a size-proportional Venn diagram
def plot_venn(unique_a, unique_b, shared_peaks, total_a, total_b, labels, colors,
              perc_a_unique, perc_a_shared, perc_b_unique, perc_b_shared):
    plt.figure(figsize=(8, 6))
    # Create the Venn diagram using the counts
    venn = venn2(subsets=(unique_a, unique_b, shared_peaks),
                 set_labels=('', ''), set_colors=colors, alpha=0.7)

    # Add black outlines to each circle
    for patch in venn.patches:
        if patch:
            patch.set_edgecolor('black')
            patch.set_linewidth(1)

    # Display total peak counts (positioned roughly on top of each circle)
    plt.text(-0.7, 0.6, f"Total: {total_a}", fontsize=10, ha='center')
    plt.text(0.7, 0.6, f"Total: {total_b}", fontsize=10, ha='center')

    # Display percentages (positioned in the bottom half)
    plt.text(-0.6, -0.6, f"{perc_a_unique:.2f}% Unique", fontsize=10, ha='center', color=colors[0])
    plt.text(0.6, -0.6, f"{perc_b_unique:.2f}% Unique", fontsize=10, ha='center', color=colors[1])
    plt.text(-0.6, -0.65, f"{perc_a_shared:.2f}% Shared with {labels[1]}", fontsize=10, ha='center', color=colors[0])
    plt.text(0.6, -0.65, f"{perc_b_shared:.2f}% Shared with {labels[0]}", fontsize=10, ha='center', color=colors[1])

    # Add labels for each TF (customize as needed)
    plt.text(-0.5, 0.5, labels[0], fontsize=12, ha='center', fontweight='bold', color=colors[0])
    plt.text(0.5, 0.5, labels[1], fontsize=12, ha='center', fontweight='bold', color=colors[1])

    plt.title("ChIP Peak Overlap", fontsize=16) #--------------------------

    # Save the figure if desired (uncomment and modify the path)
    #save_path = '/content/drive/MyDrive/ChIP-Venn/R1_SOX2_R1881-vs-PrEC_SOX2_peakVenn.png' #------------------------------------------------------------------------------------------------------
    #plt.savefig(save_path, dpi=300, bbox_inches='tight')
    #print(f"Figure saved to Google Drive at {save_path}")

    # Show the plot
    plt.show()

# Define labels and colors (adjust as needed)
#GREEN #2e6f40-dark #339c5e-kelly #32CD32-lime / BLUE #0000ff-dark #87cefa-light
#RED #ff0000-normal or #ff7f7f-light / PURPLE #cc8899-light or #6c3baa-dark
#GRAY #828282-light or #48494B-dark / PINK #FA8072-salmon / ORANGE #FFA500
labels = ["R1-AR-CSS", "PREC-SOX2"] #-------------------------------------------------------------------------------------------------------------------------------------------------
colors = ["#6c3baa", "#cc8899"] #-------------------------------------------------------------------------------------------------------------------------------------------

# Plot the Venn diagram
plot_venn(unique_a, unique_b, shared_peaks, total_a, total_b,
          labels, colors, perc_a_unique, perc_a_shared, perc_b_unique, perc_b_shared)


#3-way

In [ ]:
#no advanced percentages, only unique

# Upload BED files for three transcription factors
# IMPORTANT: Upload the raw peaks files for TF-A, TF-B, and TF-C in that order.
uploaded = files.upload()
bed_files = list(uploaded.keys())  # Order: TF-A, TF-B, TF-C

# Load BED files as BedTool objects and sort them
a_bed = BedTool(bed_files[0]).sort()
b_bed = BedTool(bed_files[1]).sort()
c_bed = BedTool(bed_files[2]).sort()

# Calculate unique regions and intersections:
# Regions unique to A: peaks in A with no overlap in B or C
only_a = a_bed.intersect(b_bed, v=True).intersect(c_bed, v=True)
# Regions unique to B:
only_b = b_bed.intersect(a_bed, v=True).intersect(c_bed, v=True)
# Regions unique to C:
only_c = c_bed.intersect(a_bed, v=True).intersect(b_bed, v=True)

# Regions shared by A and B only (not overlapping C)
a_and_b_only = a_bed.intersect(b_bed, u=True).intersect(c_bed, v=True)
# Regions shared by A and C only (not overlapping B)
a_and_c_only = a_bed.intersect(c_bed, u=True).intersect(b_bed, v=True)
# Regions shared by B and C only (not overlapping A)
b_and_c_only = b_bed.intersect(c_bed, u=True).intersect(a_bed, v=True)

# Regions shared by all three:
a_and_b_and_c = a_bed.intersect(b_bed, u=True).intersect(c_bed, u=True)

# Count peaks in each category
count_only_a = only_a.count()
count_only_b = only_b.count()
count_only_c = only_c.count()
count_ab = a_and_b_only.count()
count_ac = a_and_c_only.count()
count_bc = b_and_c_only.count()
count_abc = a_and_b_and_c.count()

# Calculate total peaks per TF
total_a = count_only_a + count_ab + count_ac + count_abc
total_b = count_only_b + count_ab + count_bc + count_abc
total_c = count_only_c + count_ac + count_bc + count_abc

# Print counts for validation
print("Counts:")
print(f"TF-A only: {count_only_a}")
print(f"TF-B only: {count_only_b}")
print(f"TF-C only: {count_only_c}")
print(f"TF-A and TF-B only: {count_ab}")
print(f"TF-A and TF-C only: {count_ac}")
print(f"TF-B and TF-C only: {count_bc}")
print(f"TF-A, TF-B, and TF-C shared: {count_abc}")
print(f"Total TF-A peaks: {total_a}")
print(f"Total TF-B peaks: {total_b}")
print(f"Total TF-C peaks: {total_c}")

# (Optional) Calculate percentages for additional info
perc_a_unique = (count_only_a / total_a) * 100 if total_a else 0
perc_b_unique = (count_only_b / total_b) * 100 if total_b else 0
perc_c_unique = (count_only_c / total_c) * 100 if total_c else 0

# Plot a size-proportional 3-way Venn diagram
def plot_venn3(count_only_a, count_only_b, count_ab, count_only_c, count_ac, count_bc, count_abc,
               total_a, total_b, total_c, labels, colors,
               perc_a_unique, perc_b_unique, perc_c_unique):
    plt.figure(figsize=(10, 8))
    # The order of subsets for venn3 is:
    # (A only, B only, A&B only, C only, A&C only, B&C only, A&B&C)
    venn = venn3(subsets=(count_only_a, count_only_b, count_ab,
                           count_only_c, count_ac, count_bc, count_abc),
                 set_labels=(labels[0], labels[1], labels[2]),
                 set_colors=colors, alpha=0.7)

    # Add black outlines to each circle
    for patch in venn.patches:
        if patch:
            patch.set_edgecolor('black')
            patch.set_linewidth(1)

    # Display total peak counts (positioned around the circles)
    plt.text(-0.75, 0.65, f"Total: {total_a}", fontsize=10, ha='center', color=colors[0])
    plt.text(0.75, 0.65, f"Total: {total_b}", fontsize=10, ha='center', color=colors[1])
    plt.text(0, -0.75, f"Total: {total_c}", fontsize=10, ha='center', color=colors[2])

    # Display unique percentages directly below the total counts:
    plt.text(-0.75, 0.55, f"{perc_a_unique:.2f}% Unique", fontsize=10, ha='center', color=colors[0])
    plt.text(0.75, 0.55, f"{perc_b_unique:.2f}% Unique", fontsize=10, ha='center', color=colors[1])
    plt.text(0, -0.85, f"{perc_c_unique:.2f}% Unique", fontsize=10, ha='center', color=colors[2])

    plt.title("(Whole Media) CWR-R1 ChIP Peak Overlap (3-way)", fontsize=16) #-----------------------------------------------------------------------------------------

    # Save the figure if desired (uncomment and adjust the save path)
    #save_path = '/content/drive/MyDrive/ChIP-Venn/TF3_Peak_Venn.png' #----------------------------------------------------------------------------
    #plt.savefig(save_path, dpi=300, bbox_inches='tight')
    #print(f"Figure saved to Google Drive at {save_path}")

    # Show the plot
    plt.show()

# Define labels and colors (adjust as needed)
#GREEN #2e6f40-dark #339c5e-kelly #32CD32-lime / BLUE #0000ff-dark #87cefa-light
#RED #ff0000-normal or #ff7f7f-light / PURPLE #cc8899-light or #6c3baa-dark
#GRAY #828282-light or #48494B-dark / PINK #FA8072-salmon / ORANGE #FFA500
labels = ["AR", "FOXA1", "SOX2"] #-------------------------------------------------------------------------------------------------------------------
# Example colors: lime green, light purple, and light blue
colors = ["#339c5e", "#ff0000", "#0000ff"] #----------------------------------------------------------------------------------------------------------

# Plot the Venn diagram
plot_venn3(count_only_a, count_only_b, count_ab, count_only_c, count_ac, count_bc, count_abc,
           total_a, total_b, total_c, labels, colors,
           perc_a_unique, perc_b_unique, perc_c_unique)


In [ ]:
#3way

#Upload BED files for three transcription factors
uploaded = files.upload()
bed_files = list(uploaded.keys())  # Order: TF-A, TF-B, TF-C

# Load BED files as BedTool objects and sort them
a_bed = BedTool(bed_files[0]).sort()
b_bed = BedTool(bed_files[1]).sort()
c_bed = BedTool(bed_files[2]).sort()

# Calculate unique regions and intersections
only_a = a_bed.intersect(b_bed, v=True).intersect(c_bed, v=True)
only_b = b_bed.intersect(a_bed, v=True).intersect(c_bed, v=True)
only_c = c_bed.intersect(a_bed, v=True).intersect(b_bed, v=True)

a_and_b_only = a_bed.intersect(b_bed, u=True).intersect(c_bed, v=True)
a_and_c_only = a_bed.intersect(c_bed, u=True).intersect(b_bed, v=True)
b_and_c_only = b_bed.intersect(c_bed, u=True).intersect(a_bed, v=True)

a_and_b_and_c = a_bed.intersect(b_bed, u=True).intersect(c_bed, u=True)

# Count peaks in each category
count_only_a = only_a.count()
count_only_b = only_b.count()
count_only_c = only_c.count()
count_ab = a_and_b_only.count()
count_ac = a_and_c_only.count()
count_bc = b_and_c_only.count()
count_abc = a_and_b_and_c.count()

# Calculate total peaks per TF
total_a = count_only_a + count_ab + count_ac + count_abc
total_b = count_only_b + count_ab + count_bc + count_abc
total_c = count_only_c + count_ac + count_bc + count_abc

# Calculate percentages
perc_a_unique = (count_only_a / total_a) * 100 if total_a else 0
perc_b_unique = (count_only_b / total_b) * 100 if total_b else 0
perc_c_unique = (count_only_c / total_c) * 100 if total_c else 0

perc_ab = (count_ab / total_a) * 100 if total_a else 0
perc_ac = (count_ac / total_a) * 100 if total_a else 0
perc_bc = (count_bc / total_b) * 100 if total_b else 0
perc_abc = (count_abc / total_a) * 100 if total_a else 0

# Plot a size-proportional 3-way Venn diagram
def plot_venn3(count_only_a, count_only_b, count_ab, count_only_c, count_ac, count_bc, count_abc,
               total_a, total_b, total_c, labels, colors,
               perc_a_unique, perc_b_unique, perc_c_unique,
               perc_ab, perc_ac, perc_bc, perc_abc):
    plt.figure(figsize=(8, 6))
    venn = venn3(subsets=(count_only_a, count_only_b, count_ab,
                           count_only_c, count_ac, count_bc, count_abc),
                 set_labels=(labels[0], labels[1], labels[2]),
                 set_colors=colors, alpha=0.7)

    for patch in venn.patches:
        if patch:
            patch.set_edgecolor('black')
            patch.set_linewidth(1)

    # Adjust labels to match bubble colors
    for i, label in enumerate(venn.set_labels):
        label.set_color(colors[i])

    # Display total peaks and percentages near the bubbles
    plt.text(-0.6, -0.6, f"Total: {total_a}\n{perc_a_unique:.1f}% Unique\n{perc_ab:.1f}% AB\n{perc_ac:.1f}% AC",
             fontsize=9, ha='center', color=colors[0])
    plt.text(0.6, 0.5, f"Total: {total_b}\n{perc_b_unique:.1f}% Unique\n{perc_ab:.1f}% AB\n{perc_bc:.1f}% BC",
             fontsize=9, ha='center', color=colors[1])
    plt.text(0.6, -0.6, f"Total: {total_c}\n{perc_c_unique:.1f}% Unique\n{perc_ac:.1f}% AC\n{perc_bc:.1f}% BC",
             fontsize=9, ha='center', color=colors[2])

    plt.title("CWR-R1 SOX2 ChIP Peak Overlap", fontsize=14) #--------------------------------------------------------------------
    plt.tight_layout()

    # Save the figure if desired (uncomment and adjust the save path)
    save_path = '/content/drive/MyDrive/ChIP-Venn/3-SOX2_Whole-SOX2_Enza-SOX2_R1881_R1.png' #----------------------------------------------------------------------------
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Figure saved to Google Drive at {save_path}")

    plt.show()

# Define labels and colors
#GREEN #2e6f40-dark #339c5e-kelly #32CD32-lime / BLUE #0000ff-dark #87cefa-light
#RED #ff0000-normal or #ff7f7f-light / PURPLE #cc8899-light or #6c3baa-dark
#GRAY #828282-light or #48494B-dark / PINK #FA8072-salmon / ORANGE #FFA500
labels = ["SOX2-Whole", "SOX2-Enza", "SOX2-R1881"] #-----------------------------------------------------------------------------------------------------------------
colors = ["#87cefa", "#ff7f7f", "#32CD32"] #--------------------------------------------------------------------------------------------------------

# Plot the Venn diagram
plot_venn3(count_only_a, count_only_b, count_ab, count_only_c, count_ac, count_bc, count_abc,
           total_a, total_b, total_c, labels, colors,
           perc_a_unique, perc_b_unique, perc_c_unique,
           perc_ab, perc_ac, perc_bc, perc_abc)